In [ ]:
segments_folder = '../matting_human_half/segments/'

In [8]:
import os
import random
import numpy as np
import cv2
import matplotlib.pyplot as plt
import sys
#sys.path.append('../')

## 测试生成trimap

In [2]:
import pickle
with open('../imgList.pkl', 'rb') as f:
    img_list = pickle.load(f)
with open('../imgMatte.pkl', 'rb') as f:
    alpha_list = pickle.load(f)

In [1]:
trimap_folder = '../matting_human_half/segments/trimap'
image_folder = '../matting_human_half/segments/image'
alpha_folder = '../matting_human_half/segments/alpha'

In [ ]:
img = plt.imread('../'+img_list[0])

In [ ]:
prefix_name = img_list[0].split('/')[-1].split('.')[0]
for alpha_file in alpha_list:
    if prefix_name in alpha_file:
        alpha = plt.imread('../'+alpha_file)
        break

In [ ]:
plt.imshow(((alpha[:, :, -1] < 0.8) & (alpha[:, :, -1] > 0.1)).astype(np.int32) * 255)

In [6]:
def gen_trimap(alpha, fg_thr=0.8, bg_thr=0.1, kernel_max=15, kernel_min=5):
    if len(alpha.shape) == 3 and alpha.shape[-1] == 4:
        alpha = alpha[:, :, -1].copy()
    trimap = np.zeros(alpha.shape)
    unsure_msk = np.zeros(alpha.shape)
    trimap[alpha >= fg_thr] = 255
    trimap[alpha <= bg_thr] = 0
    
    unsure_msk[(alpha>bg_thr) & (alpha<fg_thr)] = 1
    kernel_size = random.randint(kernel_min, kernel_max)
    print(kernel_size)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))
    unsure_msk = cv2.dilate(unsure_msk, kernel, iterations=1).astype(bool)
    
    trimap[unsure_msk] = 128
    return trimap

In [ ]:
test = plt.imread('test.png')
test.shape

In [ ]:
for alpha_file in alpha_list:
    filename = alpha_file.split('/')[-1]
    if not os.path.isfile('../'+alpha_file):
        print(alpha_file, 'missing')
        continue
    alpha = cv2.imread('../'+alpha_file, -1)
    cv2.imwrite(image_folder+'/'+filename, alpha[:, :, :3])
    #cv2.imwrite(alpha_folder+'/'+filename, alpha[:, :, -1])
    #cv2.imwrite(trimap_folder+'/'+filename, gen_trimap(alpha))

In [ ]:
alpha_list[0]

In [ ]:
exist = os.listdir('../matting_human_half/segments/alpha')

In [ ]:
remove_list = []
for f in alpha_list:
    if f.split('/')[-1] in exist:
        remove_list.append(f)
for f in remove_list:
    alpha_list.remove(f)